In [ ]:
%sh
sudo ACCEPT_EULA=Y apt-get install odbcinst
sudo curl https://packages.microsoft.com/keys/microsoft.asc | sudo apt-key add -
echo "deb [arch=amd64] https://packages.microsoft.com/ubuntu/21.10/prod impish main" | sudo tee /etc/apt/sources.list.d/mssql-release.list
sudo apt update
sudo ACCEPT_EULA=Y apt install msodbcsql18

In [ ]:
pip install sqlalchemy

In [ ]:
dbutils.library.restartPython() 

In [ ]:
import requests
import zipfile
import io

# URL of the ZIP file
url = 'https://www.sec.gov/Archives/edgar/daily-index/xbrl/companyfacts.zip'

# Send a GET request to the URL
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
# Extract the content from the response
zip_content = io.BytesIO(response.content)
# Create a ZipFile object
with zipfile.ZipFile(zip_content, 'r') as zip_ref:
    # Extract all the contents to a specified directory in Databricks File System (DBFS)
    zip_ref.extractall('/dbfs/mnt/out/extracted_files/')  
print("Extraction completed successfully.")
# Copy the contents of the extracted files directory in DBFS to Azure Data Lake Storage Gen2
dbutils.fs.cp('dbfs:/mnt/out/extracted_files/','abfss://project@firstdemoto.dfs.core.windows.net/jsonfiles/',recurse=True)


In [ ]:
l1=spark.createDataFrame(dbutils.fs.ls('abfss://project@firstdemoto.dfs.core.windows.net/jsonfiles/'))
rdd1=l1.select('path').rdd
rdd1=rdd1.map(lambda x:x[0])
file_list=rdd1.collect()

In [ ]:
import json
import pandas as pd
from pyspark import SparkConf, SparkContext
from sqlalchemy import create_engine
import pyodbc

conf = SparkConf().setAppName('json')
spc = SparkContext.getOrCreate(conf=conf)

for file in file_list:
    print('Processing file:',file)
    rdd = spc.textFile(file)
    dei_df=pd.DataFrame([])
    invest_df=pd.DataFrame([])
    srt_df=pd.DataFrame([])
    if rdd.count() == 0:
        print(rdd.count())
        print(rdd.collect())
    else:
        l1 = rdd.collect()
        d1 = json.loads(l1[0])
        df = pd.DataFrame(d1)

        if 'facts' in df.columns and 'cik' in df.columns:
            try:
                df = df.drop(['us-gaap'], axis=0)
            except:
                pass
            x = dict(df['facts'])
            try:
                del x['us-gaap']
            except:
                pass
            key = list(x.keys())
            value = list(x.values())
            li1 = []

            for i in range(0, len(value)):
                key1 = list(value[i].keys())
                value1 = list(value[i].values())
                li = []
                for j in range(0, len(value1)):
                    key2 = list(value1[j].keys())
                    value2 = list(value1[j].values())
                    sh = value2[2]
                    key3 = list(sh.keys())
                    value3 = list(sh.values())
                    l1 = key3
                    l2 = [value3[0][-1]]
                    d1 = dict(zip(l1, l2))
                    value2[2] = d1
                    d2 = dict(zip(key2, value2))
                    li.append(d2)
                d3 = dict(zip(key1, li))
                li1.append(d3)
            d4 = dict(zip(key, li1))

            try:
                df = df.drop(['us-gaap'], axis=0)
            except:
                pass
            df['facts'] = d4.values()
            d2 = df['facts']
            cik_number = df.cik.unique()
            entityName = df.entityName.unique()
            driver = 'Driver={ODBC Driver 18 for SQL Server};Server=tcp:capprodbserver.database.windows.net,1433;Database=cappro;Uid=des;Pwd=Suresh@1986;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'
            engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(driver))
            check_keys = df['facts'].keys()
            if 'dei' in check_keys:
                dei = pd.json_normalize(df['facts']['dei'])
                lis_col1=['EntityCommonStockSharesOutstanding.label'
                    ,'EntityCommonStockSharesOutstanding.units.shares.end'
                    ,'EntityCommonStockSharesOutstanding.units.shares.val'
                    ,'EntityCommonStockSharesOutstanding.units.shares.accn'
                    ,'EntityCommonStockSharesOutstanding.units.shares.fy'
                    ,'EntityCommonStockSharesOutstanding.units.shares.fp'
                    ,'EntityCommonStockSharesOutstanding.units.shares.form'
                    ,'EntityCommonStockSharesOutstanding.units.shares.filed'
                    ,'EntityCommonStockSharesOutstanding.units.shares.frame'
                    ,'EntityListingParValuePerShare.label'
                    ,'EntityListingParValuePerShare.units.USD/shares.start'
                    ,'EntityListingParValuePerShare.units.USD/shares.end'
                    ,'EntityListingParValuePerShare.units.USD/shares.val'
                    ,'EntityListingParValuePerShare.units.USD/shares.accn'
                    ,'EntityListingParValuePerShare.units.USD/shares.fy'
                    ,'EntityListingParValuePerShare.units.USD/shares.fp'
                    ,'EntityListingParValuePerShare.units.USD/shares.form'
                    ,'EntityListingParValuePerShare.units.USD/shares.filed'
                    ,'EntityPublicFloat.label'
                    ,'EntityPublicFloat.units.USD.end'
                    ,'EntityPublicFloat.units.USD.val'
                    ,'EntityPublicFloat.units.USD.accn'
                    ,'EntityPublicFloat.units.USD.fy'
                    ,'EntityPublicFloat.units.USD.fp'
                    ,'EntityPublicFloat.units.USD.form'
                    ,'EntityPublicFloat.units.USD.filed'
                    ,'EntityPublicFloat.units.USD.frame']
                for i in lis_col1:
                    if i in dei.columns:
                        dei_df[i]=dei[i]
                    else:
                        dei_df[i]='Null'      
                        
                dd3 = pd.DataFrame({'cik': cik_number, 'entityName': entityName})
                dei_df = dd3.merge(dei_df, how='cross')
            
                # Rename columns with invalid characters
                dei_df.columns = dei_df.columns.str.replace('/', '_')
                dei_df.columns = dei_df.columns.str.replace("'", "''")
                dei_df.to_sql('dei', schema='dbo', con=engine, chunksize=200, method='multi', index=False, if_exists='append')
            if 'invest' in check_keys:

                invest = pd.json_normalize(df['facts']['invest'])
                lis_col2=['InvestmentInterestRateRangeEnd.label'
                        ,'InvestmentInterestRateRangeEnd.units.pure.start'
                        ,'InvestmentInterestRateRangeEnd.units.pure.end'
                        ,'InvestmentInterestRateRangeEnd.units.pure.val'
                        ,'InvestmentInterestRateRangeEnd.units.pure.accn'
                        ,'InvestmentInterestRateRangeEnd.units.pure.fy'
                        ,'InvestmentInterestRateRangeEnd.units.pure.fp'
                        ,'InvestmentInterestRateRangeEnd.units.pure.form'
                        ,'InvestmentInterestRateRangeEnd.units.pure.filed'
                        ,'InvestmentInterestRateRangeEnd.units.pure.frame'
                        ,'InvestmentWarrantsExercisePrice.label'
                        ,'InvestmentWarrantsExercisePrice.units.USD.start'
                        ,'InvestmentWarrantsExercisePrice.units.USD.end'
                        ,'InvestmentWarrantsExercisePrice.units.USD.val'
                        ,'InvestmentWarrantsExercisePrice.units.USD.accn'
                        ,'InvestmentWarrantsExercisePrice.units.USD.fy'
                        ,'InvestmentWarrantsExercisePrice.units.USD.fp'
                        ,'InvestmentWarrantsExercisePrice.units.USD.form'
                        ,'InvestmentWarrantsExercisePrice.units.USD.filed'
                        ]
                for i in lis_col2:
                    if i in invest.columns:
                        invest_df[i]=invest[i]
                    else:
                        invest_df[i]='Null'      
                invest_df = dd3.merge(invest_df, how='cross')
                
                # Rename columns with invalid characters
                invest_df.columns = invest_df.columns.str.replace('/', '_')
                invest_df.columns = invest_df.columns.str.replace("'", "'''")
                invest_df.to_sql('invest', schema='dbo', con=engine, chunksize=200, method='multi', index=False, if_exists='append')
            if 'srt' in check_keys:
                srt = pd.json_normalize(df['facts']['srt'])
                lis_col3=['ExtensionsDiscoveriesAdditionsAndImprovedRecoveryLessRelatedCosts.label'
                        ,'ExtensionsDiscoveriesAdditionsAndImprovedRecoveryLessRelatedCosts.units.USD.accn'
                        ,'ExtensionsDiscoveriesAdditionsAndImprovedRecoveryLessRelatedCosts.units.USD.end'
                        ,'ExtensionsDiscoveriesAdditionsAndImprovedRecoveryLessRelatedCosts.units.USD.filed'
                        ,'ExtensionsDiscoveriesAdditionsAndImprovedRecoveryLessRelatedCosts.units.USD.form'
                        ,'ExtensionsDiscoveriesAdditionsAndImprovedRecoveryLessRelatedCosts.units.USD.fp'
                        ,'ExtensionsDiscoveriesAdditionsAndImprovedRecoveryLessRelatedCosts.units.USD.frame'
                        ,'ExtensionsDiscoveriesAdditionsAndImprovedRecoveryLessRelatedCosts.units.USD.fy'
                        ,'ExtensionsDiscoveriesAdditionsAndImprovedRecoveryLessRelatedCosts.units.USD.start'
                        ,'ExtensionsDiscoveriesAdditionsAndImprovedRecoveryLessRelatedCosts.units.USD.val'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesCashInflows.label'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesCashInflows.units.USD.accn'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesCashInflows.units.USD.end'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesCashInflows.units.USD.filed'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesCashInflows.units.USD.form'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesCashInflows.units.USD.fp'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesCashInflows.units.USD.frame'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesCashInflows.units.USD.fy'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesCashInflows.units.USD.val'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesDevelopmentCosts.label'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesDevelopmentCosts.units.USD.accn'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesDevelopmentCosts.units.USD.end'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesDevelopmentCosts.units.USD.filed'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesDevelopmentCosts.units.USD.form'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesDevelopmentCosts.units.USD.fp'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesDevelopmentCosts.units.USD.frame'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesDevelopmentCosts.units.USD.fy'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesDevelopmentCosts.units.USD.val'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesIncomeTaxExpense.label'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesIncomeTaxExpense.units.USD.accn'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesIncomeTaxExpense.units.USD.end'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesIncomeTaxExpense.units.USD.filed'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesIncomeTaxExpense.units.USD.form'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesIncomeTaxExpense.units.USD.fp'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesIncomeTaxExpense.units.USD.frame'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesIncomeTaxExpense.units.USD.fy'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesIncomeTaxExpense.units.USD.val'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesNetCashFlows.label'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesNetCashFlows.units.USD.accn'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesNetCashFlows.units.USD.end'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesNetCashFlows.units.USD.filed'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesNetCashFlows.units.USD.form'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesNetCashFlows.units.USD.fp'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesNetCashFlows.units.USD.frame'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesNetCashFlows.units.USD.fy'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesNetCashFlows.units.USD.val'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesTenPercentAnnualDiscountForEstimatedTimingOfCashFlows.label'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesTenPercentAnnualDiscountForEstimatedTimingOfCashFlows.units.USD.accn'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesTenPercentAnnualDiscountForEstimatedTimingOfCashFlows.units.USD.end'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesTenPercentAnnualDiscountForEstimatedTimingOfCashFlows.units.USD.filed'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesTenPercentAnnualDiscountForEstimatedTimingOfCashFlows.units.USD.form'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesTenPercentAnnualDiscountForEstimatedTimingOfCashFlows.units.USD.fp'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesTenPercentAnnualDiscountForEstimatedTimingOfCashFlows.units.USD.frame'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesTenPercentAnnualDiscountForEstimatedTimingOfCashFlows.units.USD.fy'
                        ,'FutureNetCashFlowsRelatingToProvedOilAndGasReservesTenPercentAnnualDiscountForEstimatedTimingOfCashFlows.units.USD.val'
                        ,'NetIncreaseDecreaseInSalesAndTransferPricesAndProductionCosts.label'
                        ,'NetIncreaseDecreaseInSalesAndTransferPricesAndProductionCosts.units.USD.accn'
                        ,'NetIncreaseDecreaseInSalesAndTransferPricesAndProductionCosts.units.USD.end'
                        ,'NetIncreaseDecreaseInSalesAndTransferPricesAndProductionCosts.units.USD.filed'
                        ,'NetIncreaseDecreaseInSalesAndTransferPricesAndProductionCosts.units.USD.form'
                        ,'NetIncreaseDecreaseInSalesAndTransferPricesAndProductionCosts.units.USD.fp'
                        ,'NetIncreaseDecreaseInSalesAndTransferPricesAndProductionCosts.units.USD.frame'
                        ,'NetIncreaseDecreaseInSalesAndTransferPricesAndProductionCosts.units.USD.fy'
                        ,'NetIncreaseDecreaseInSalesAndTransferPricesAndProductionCosts.units.USD.start'
                        ,'NetIncreaseDecreaseInSalesAndTransferPricesAndProductionCosts.units.USD.val'
                        ,'RevisionsOfPreviousQuantityEstimates.label'
                        ,'RevisionsOfPreviousQuantityEstimates.units.USD.accn'
                        ,'RevisionsOfPreviousQuantityEstimates.units.USD.end'
                        ,'RevisionsOfPreviousQuantityEstimates.units.USD.filed'
                        ,'RevisionsOfPreviousQuantityEstimates.units.USD.form'
                        ,'RevisionsOfPreviousQuantityEstimates.units.USD.fp'
                        ,'RevisionsOfPreviousQuantityEstimates.units.USD.frame'
                        ,'RevisionsOfPreviousQuantityEstimates.units.USD.fy'
                        ,'RevisionsOfPreviousQuantityEstimates.units.USD.start'
                        ,'RevisionsOfPreviousQuantityEstimates.units.USD.val'
                        ,'SalesAndTransfersOfOilAndGasProducedNetOfProductionCosts.label'
                        ,'SalesAndTransfersOfOilAndGasProducedNetOfProductionCosts.units.USD.accn'
                        ,'SalesAndTransfersOfOilAndGasProducedNetOfProductionCosts.units.USD.end'
                        ,'SalesAndTransfersOfOilAndGasProducedNetOfProductionCosts.units.USD.filed'
                        ,'SalesAndTransfersOfOilAndGasProducedNetOfProductionCosts.units.USD.form'
                        ,'SalesAndTransfersOfOilAndGasProducedNetOfProductionCosts.units.USD.fp'
                        ,'SalesAndTransfersOfOilAndGasProducedNetOfProductionCosts.units.USD.frame'
                        ,'SalesAndTransfersOfOilAndGasProducedNetOfProductionCosts.units.USD.fy'
                        ,'SalesAndTransfersOfOilAndGasProducedNetOfProductionCosts.units.USD.start'
                        ,'SalesAndTransfersOfOilAndGasProducedNetOfProductionCosts.units.USD.val'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowOfProvedOilAndGasReservesOther.label'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowOfProvedOilAndGasReservesOther.units.USD.accn'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowOfProvedOilAndGasReservesOther.units.USD.end'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowOfProvedOilAndGasReservesOther.units.USD.filed'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowOfProvedOilAndGasReservesOther.units.USD.form'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowOfProvedOilAndGasReservesOther.units.USD.fp'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowOfProvedOilAndGasReservesOther.units.USD.frame'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowOfProvedOilAndGasReservesOther.units.USD.fy'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowOfProvedOilAndGasReservesOther.units.USD.start'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowOfProvedOilAndGasReservesOther.units.USD.val'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowRelatingToProvedOilAndGasReservesAccretionOfDiscount.label'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowRelatingToProvedOilAndGasReservesAccretionOfDiscount.units.USD.accn'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowRelatingToProvedOilAndGasReservesAccretionOfDiscount.units.USD.end'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowRelatingToProvedOilAndGasReservesAccretionOfDiscount.units.USD.filed'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowRelatingToProvedOilAndGasReservesAccretionOfDiscount.units.USD.form'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowRelatingToProvedOilAndGasReservesAccretionOfDiscount.units.USD.fp'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowRelatingToProvedOilAndGasReservesAccretionOfDiscount.units.USD.frame'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowRelatingToProvedOilAndGasReservesAccretionOfDiscount.units.USD.fy'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowRelatingToProvedOilAndGasReservesAccretionOfDiscount.units.USD.start'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowRelatingToProvedOilAndGasReservesAccretionOfDiscount.units.USD.val'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowsRelatingToProvedOilAndGasReserves.label'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowsRelatingToProvedOilAndGasReserves.units.USD.accn'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowsRelatingToProvedOilAndGasReserves.units.USD.end'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowsRelatingToProvedOilAndGasReserves.units.USD.filed'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowsRelatingToProvedOilAndGasReserves.units.USD.form'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowsRelatingToProvedOilAndGasReserves.units.USD.fp'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowsRelatingToProvedOilAndGasReserves.units.USD.frame'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowsRelatingToProvedOilAndGasReserves.units.USD.fy'
                        ,'StandardizedMeasureOfDiscountedFutureNetCashFlowsRelatingToProvedOilAndGasReserves.units.USD.val'
                        ]
                for i in lis_col3:
                    if i in srt.columns:
                        srt_df[i]=srt[i]
                    else:
                        srt_df[i]='Null'  
                srt_df = dd3.merge(srt_df, how='cross')

                # Rename columns with invalid characters
                srt_df.columns = srt_df.columns.str.replace('/', '_')
                srt_df.columns = srt_df.columns.str.replace("'", "''")
                srt_df.to_sql('srt', schema='dbo', con=engine, chunksize=200, method='multi', index=False, if_exists='append')

        print('Insert Dataframe into SQL Server:')